In [1]:
# Importation des bibliothèques nécessaires
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
# 1. Chargement des données prétraitées
df = pd.read_csv('preprocessed_data.csv')

In [3]:
# 2. Sélection des caractéristiques (features) et de la variable cible (target)
X = df.drop(columns=['resale_price'])  # Caractéristiques
y = df['resale_price']  # Variable cible

In [4]:
# 3. Gestion des colonnes non numériques
# Convertir les colonnes de type date en format numérique (par exemple, années ou jours depuis une date de référence)
if 'lease_commence_date' in X.columns:
    X['lease_commence_date'] = pd.to_datetime(X['lease_commence_date'], errors='coerce')
    X['lease_commence_date'] = (X['lease_commence_date'] - X['lease_commence_date'].min()) / pd.Timedelta(days=1)  # Nombre de jours depuis la première date

# Encoder les variables catégorielles si nécessaire
for col in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

In [5]:
# 4. Division des données en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Normalisation des données
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [6]:
# 6. Modèle de régression linéaire
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Prédictions avec le modèle de régression linéaire
y_pred_lr = lr_model.predict(X_test_scaled)

# Évaluation du modèle de régression linéaire
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae_lr = mean_absolute_error(y_test, y_pred_lr)
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("Régression Linéaire - Évaluation :")
print(f"MAE: {mae_lr}")
print(f"MSE: {mse_lr}")
print(f"R2: {r2_lr}")

Régression Linéaire - Évaluation :
MAE: 72928.70521472645
MSE: 10448395308.479977
R2: 0.6326645568368452


In [7]:
# 2. Entraînement du modèle XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
model.fit(X_train_scaled, y_train)

# 3. Prédiction
y_pred = model.predict(X_test_scaled)

# 4. Évaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 5. Affichage des résultats
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")

MAE: 40151.03920628834
MSE: 3579235005.6439905
R²: 0.8741644206439788


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, LabelEncoder
import mlflow
import mlflow.sklearn

In [9]:
# 6. Définir et configurer MLflow
mlflow.set_experiment("First experiment")
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

# Modèle de régression linéaire
with mlflow.start_run(run_name="Linear Regression"):
    # Entraînement du modèle
    lr_model = LinearRegression()
    lr_model.fit(X_train_scaled, y_train)

    # Prédictions
    y_pred_lr = lr_model.predict(X_test_scaled)

    # Évaluation
    mae_lr = mean_absolute_error(y_test, y_pred_lr)
    mse_lr = mean_squared_error(y_test, y_pred_lr)
    r2_lr = r2_score(y_test, y_pred_lr)

    # Enregistrement dans MLflow
    mlflow.log_params({"model_type": "LinearRegression"})
    mlflow.log_metrics({"mae": mae_lr, "mse": mse_lr, "r2": r2_lr})
    mlflow.sklearn.log_model(lr_model, "linear_regression_model")

    # Affichage des résultats
    print("Régression Linéaire - Évaluation :")
    print(f"MAE: {mae_lr}")
    print(f"MSE: {mse_lr}")
    print(f"R2: {r2_lr}")



MlflowException: API request to http://127.0.0.1:5000/api/2.0/mlflow/runs/create failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/create (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022E2C002B90>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))

In [ ]:
# Modèle XGBoost
with mlflow.start_run(run_name="XGBoost"):
    # Entraînement du modèle
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
    xgb_model.fit(X_train_scaled, y_train)

    # Prédictions
    y_pred_xgb = xgb_model.predict(X_test_scaled)

    # Évaluation
    mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)

    # Enregistrement dans MLflow
    mlflow.log_params({
        "model_type": "XGBoost",
        "n_estimators": 100,
        "learning_rate": 0.1,
        "max_depth": 5
    })
    mlflow.log_metrics({"mae": mae_xgb, "mse": mse_xgb, "r2": r2_xgb})
    mlflow.sklearn.log_model(xgb_model, "xgboost_model")

    # Affichage des résultats
    print("\nXGBoost - Évaluation :")
    print(f"MAE: {mae_xgb}")
    print(f"MSE: {mse_xgb}")
    print(f"R2: {r2_xgb}")

2024/11/29 19:01:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



XGBoost - Évaluation :
MAE: 40151.03920628834
MSE: 3579235005.6439905
R2: 0.8741644206439788
🏃 View run XGBoost at: http://127.0.0.1:5000/#/experiments/130527551246863566/runs/89863fc1667e48dd912315cf844b18f8
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/130527551246863566


In [ ]:
# 7. Register and log the Linear Regression model
with mlflow.start_run(run_name="Linear Regression with SMOTE"):
    # Train the model
    lr_model = LinearRegression()
    lr_model.fit(X_train_scaled, y_train)

    # Predictions and evaluation
    y_pred_lr = lr_model.predict(X_test_scaled)
    mae_lr = mean_absolute_error(y_test, y_pred_lr)
    mse_lr = mean_squared_error(y_test, y_pred_lr)
    r2_lr = r2_score(y_test, y_pred_lr)

    # Log parameters and metrics
    mlflow.log_params({"model_type": "LinearRegression"})
    mlflow.log_metrics({"mae": mae_lr, "mse": mse_lr, "r2": r2_lr})

    # Log and register the model
    mlflow.sklearn.log_model(lr_model, "linear_regression_model_with_smote")
    result = mlflow.register_model(
        "runs:/{}/linear_regression_model_with_smote".format(mlflow.active_run().info.run_id),
        "LinearRegression_SMOTE"
    )
    print("Model registered successfully with version:", result.version)

# 8. Register and log the XGBoost model
with mlflow.start_run(run_name="XGBoost with SMOTE"):
    # Train the model
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
    xgb_model.fit(X_train_scaled, y_train)

    # Predictions and evaluation
    y_pred_xgb = xgb_model.predict(X_test_scaled)
    mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)

    # Log parameters and metrics
    mlflow.log_params({
        "model_type": "XGBoost",
        "n_estimators": 100,
        "learning_rate": 0.1,
        "max_depth": 5
    })
    mlflow.log_metrics({"mae": mae_xgb, "mse": mse_xgb, "r2": r2_xgb})

    # Log and register the model
    mlflow.sklearn.log_model(xgb_model, "xgboost_model_with_smote")
    result = mlflow.register_model(
        "runs:/{}/xgboost_model_with_smote".format(mlflow.active_run().info.run_id),
        "XGBoost_SMOTE"
    )
    print("Model registered successfully with version:", result.version)


NameError: name 'X_train_scaled' is not defined